In [1]:
import torch
import torch.nn as nn
import torch.utils.data as tdata
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import numpy as np
import string

In [2]:
input_size=784
hidden_size=500
num_classes=10
num_epochs=5
batch_size=100
learn_rate=0.001

train_dataset=dsets.MNIST(root='./data',train=True, transform=transforms.ToTensor(),download=True)
test_dataset=dsets.MNIST(root='./data',train=False, transform=transforms.ToTensor())
train_loader=tdata.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)
test_loader=tdata.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)
print("Size of training data="+str(len(train_dataset)))
print("Size of test data=" +str(len(test_dataset)))


Size of training data=60000
Size of test data=10000


In [3]:
class Net(nn.Module):
    def __init__(self,input_size,hidden_size,num_classes):
        super(Net,self).__init__()
        self.fcl=nn.Linear(input_size,hidden_size)
        self.relu=nn.ReLU()
        self.fc2 = nn.Linear(hidden_size,num_classes)
        
    def forward(self,x):
        out=self.fcl(x)
        out=self.relu(out)
        out=self.fc2(out)
        return out
net=Net(input_size,hidden_size,num_classes)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(net.parameters(),lr=learn_rate)
    


In [4]:
nsteps=int(len(train_dataset)/batch_size)

for epoch in range(0,num_epochs):
    for i,(images,labels) in enumerate(train_loader):
        images=Variable(images.view(-1,input_size))
        labels=Variable(labels)
        
        optimizer.zero_grad()
        outputs=net(images)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) %100==0:
            print("Epoch="+str(epoch))
            print("Step= "+str(i+1))
            print("Loss= "+str(loss.data.item()))

Epoch=0
Step= 100
Loss= 0.42389512062072754
Epoch=1
Step= 100
Loss= 0.25517237186431885
Epoch=2
Step= 100
Loss= 0.08826715499162674
Epoch=3
Step= 100
Loss= 0.12104799598455429
Epoch=4
Step= 100
Loss= 0.055754438042640686


In [5]:
total=0
correct=0

confusion_matrix=np.zeros((num_classes,num_classes))
for images,labels in test_loader:
    images=Variable(images.view(-1,input_size))
    outputs=net(images)
    _, predicted = torch.max(outputs.data, 1)
    
    for i in range(0,len(labels)):
        act=labels[i]
        prd=predicted[i]
        confusion_matrix[act,prd]+=1
    total+=labels.size(0)
    correct+=(predicted==labels).sum()
print(' ')
print('Classification Accuracy: {:.2f}%'.format(100 * correct / total))
print(' ')
print('Confusion Matrix')
print('      |----------------------- Predicted -------------------------|')
print('Actual|  0  |  1  |  2  |  3  |  4  |  5  |  6  |  7  |  8  |  9  |')
print('======|=====|=====|=====|=====|=====|=====|=====|=====|=====|=====|')
for i in range(0,10):
    print('   {:d}  |'.format(i), end='')
    for j in range(0,10):
        cm = int(confusion_matrix[i,j])
        print('{:5d}|'.format(cm),end='')
    print('')
    
torch.save(net.state_dict(),'mnist_model.pkl')

 
Classification Accuracy: 97.00%
 
Confusion Matrix
      |----------------------- Predicted -------------------------|
Actual|  0  |  1  |  2  |  3  |  4  |  5  |  6  |  7  |  8  |  9  |
======|=====|=====|=====|=====|=====|=====|=====|=====|=====|=====|
   0  |  962|    0|    0|    1|    0|    2|   10|    1|    4|    0|
   1  |    0| 1124|    3|    1|    0|    1|    1|    3|    2|    0|
   2  |    0|    1| 1006|    1|    3|    0|    3|   11|    7|    0|
   3  |    0|    0|    1|  975|    0|   14|    0|   11|    9|    0|
   4  |    0|    0|    1|    0|  963|    0|    3|    2|    3|   10|
   5  |    1|    1|    0|    2|    0|  880|    6|    0|    0|    2|
   6  |    1|    3|    0|    0|    1|   10|  941|    1|    1|    0|
   7  |    1|    4|    0|    0|    0|    1|    0| 1020|    1|    1|
   8  |    1|    1|    0|    7|    1|    5|    4|    5|  949|    1|
   9  |    3|    5|    0|    7|    8|    6|    1|    9|    5|  965|
